<a href="https://colab.research.google.com/github/fundaylncii/RecommendationSystems/blob/main/ModelBasedMatrixFactorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## kullanıcı davranışları ve film değerlelendirmeleri göz önüne alınarak kullanıcıların izlemedikleri filmlere verecekleri tepkilerin ölçümlendirilmesi işlemleridir.
## user ve movie ler için var olduğu varsayılan latent(gizli) featureların ağırlıkları var olan veri üzerinden bulunur ve bu ağırlıklar ile var olmayan gözlemler için tahmin yapılır.
## rating matrisinin iki factor matrisinin çarpımı (dot product) ile oluştuğu varsayılır.
## Factor matris: user latent factor, movie latent factor
## Latent factor: Latent Features: Gizli faktörler veya değişkenler
## kullanıcıların ve filmlerin latent factorler için skorlara sahip oldukları düşünülür
## bu ağırlıklar(skorlar) önce var olan veri üzerinden bulunur ve sonra Blank(boş) bölümler bu ağırlıklara göre doldurulur

## latent factor: film türleri, belirli bir oyuncunun olması, yönetmen vs.

## minimum hatalı latent scorlarını oluşturmak için Gradient Descent yöntemi kullanılır.

In [1]:
pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357248 sha256=eca4d523899818693e9f0a1183e57a1696ac8496b55a1dab519d3e4936049e1e
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
import pandas as pd
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate
pd.set_option("display.max_columns", None)

In [3]:
## DataSet Url: https://grouplens.org/datasets/movielens/
movie = pd.read_csv("/content/movie.csv")
rating = pd.read_csv("/content/rating.csv")
df = movie.merge(rating, how="left", on="movieId")
df.head(10)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41
5,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12.0,4.0,1997-03-22 20:48:38
6,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,4.0,1996-11-27 08:21:31
7,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14.0,4.5,2008-10-29 20:13:59
8,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,16.0,3.0,2001-05-27 13:22:52
9,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,5.0,1997-02-05 21:03:48


In [4]:
movie_id = [130219, 356, 4422, 541]
movies = ["The Dark Knight (2011)","Cries and Whispers (Viskningar och rop) (1972)","Forrest Gump (1994)","Blade Runner (1982)"]

sample_df = df[df.movieId.isin(movie_id)]
sample_df.head(10)


,movieId,title,genres,userId,rating,timestamp
451895,356,Forrest Gump (1994),Comedy|Drama|Romance|War,4.0,4.0,1996-08-24 09:28:42
451896,356,Forrest Gump (1994),Comedy|Drama|Romance|War,7.0,4.0,2002-01-16 19:02:55
451897,356,Forrest Gump (1994),Comedy|Drama|Romance|War,8.0,5.0,1996-06-05 13:44:19
451898,356,Forrest Gump (1994),Comedy|Drama|Romance|War,9.0,4.0,2001-07-01 20:26:38
451899,356,Forrest Gump (1994),Comedy|Drama|Romance|War,10.0,3.0,1999-11-25 02:32:02
451900,356,Forrest Gump (1994),Comedy|Drama|Romance|War,11.0,5.0,2009-01-02 01:13:24
451901,356,Forrest Gump (1994),Comedy|Drama|Romance|War,12.0,4.0,1997-03-22 20:53:21
451902,356,Forrest Gump (1994),Comedy|Drama|Romance|War,13.0,5.0,1996-11-27 08:13:39
451903,356,Forrest Gump (1994),Comedy|Drama|Romance|War,14.0,4.0,2008-10-29 20:16:51
451904,356,Forrest Gump (1994),Comedy|Drama|Romance|War,15.0,5.0,1996-08-16 14:50:39


In [5]:
user_movies_df = sample_df.pivot_table(index=["userId"], columns=["title"], values="rating")
user_movies_df.head(10)

title,Blade Runner (1982),Cries and Whispers (Viskningar och rop) (1972),Forrest Gump (1994),The Dark Knight (2011)
userId,,,,
1.0,4.0,NaN,NaN,NaN
2.0,5.0,NaN,NaN,NaN
3.0,5.0,NaN,NaN,NaN
4.0,NaN,NaN,4.0,NaN
7.0,NaN,NaN,4.0,NaN
8.0,NaN,NaN,5.0,NaN
9.0,NaN,NaN,4.0,NaN
10.0,NaN,NaN,3.0,NaN
11.0,4.5,NaN,5.0,NaN


In [6]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(sample_df[["userId", "movieId", "rating"]], reader)


In [7]:
trainset, testset = train_test_split(data, test_size=.25)
svd_model = SVD()
svd_model.fit(trainset)

In [8]:
## factor tahminleri oluşturuldu
prediction = svd_model.test(testset)
print(prediction)

[Prediction(uid=13028.0, iid=356, r_ui=4.0, est=4.10053067325189, details={'was_impossible': False}), Prediction(uid=16318.0, iid=356, r_ui=4.0, est=3.8601172287313767, details={'was_impossible': False}), Prediction(uid=5548.0, iid=356, r_ui=3.5, est=4.10053067325189, details={'was_impossible': False}), Prediction(uid=6593.0, iid=356, r_ui=4.0, est=4.281560718639141, details={'was_impossible': False}), Prediction(uid=3944.0, iid=356, r_ui=4.5, est=3.898013036323381, details={'was_impossible': False}), Prediction(uid=20193.0, iid=356, r_ui=3.0, est=3.790276518809941, details={'was_impossible': False}), Prediction(uid=15992.0, iid=356, r_ui=4.5, est=4.10053067325189, details={'was_impossible': False}), Prediction(uid=13410.0, iid=356, r_ui=4.0, est=4.10053067325189, details={'was_impossible': False}), Prediction(uid=16000.0, iid=541, r_ui=3.5, est=4.14898203765249, details={'was_impossible': False}), Prediction(uid=24960.0, iid=356, r_ui=2.0, est=4.10053067325189, details={'was_impossibl

In [9]:
## hata payının hesaplanması
accuracy.rmse(prediction)

RMSE: 0.9364


0.9364211459811149

In [10]:
## kullanıcı bazlı sonuç görüntüleme(est=3.92149225101374 1 nolu kullanıcının 541 nolu filme vereceği puan tahmini)
svd_model.predict(uid=1.0, iid=541, verbose=True)

user: 1.0        item: 541        r_ui = None   est = 3.92   {'was_impossible': False}


Prediction(uid=1.0, iid=541, r_ui=None, est=3.92149225101374, details={'was_impossible': False})

In [11]:
## kullanıcı bazlı sonuç görüntüleme(est=4.459217860001687 1 nolu kullanıcının 356 nolu filme vereceği puan tahmini)
svd_model.predict(uid=1.0, iid=356, verbose=True)

user: 1.0        item: 356        r_ui = None   est = 4.46   {'was_impossible': False}


Prediction(uid=1.0, iid=356, r_ui=None, est=4.459217860001687, details={'was_impossible': False})

In [14]:
param_grid = {"n_epochs": [5,10,20],
              "lr_all": [0.002, 0.005, 0.007]}

gs = GridSearchCV(SVD,
                 param_grid,
                  measures=["rmse","mae"],
                  cv=3,
                  n_jobs=-1,
                  joblib_verbose=True)

In [15]:
gs.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    5.6s finished


In [16]:
gs.best_score["rmse"]

0.9295066528652994

In [17]:
## en iyi dışsal parametre
gs.best_params["rmse"]

{'n_epochs': 5, 'lr_all': 0.002}

In [19]:
## Final Model and Prediction
svd_model = SVD(**gs.best_params["rmse"])
data = data.build_full_trainset()

In [20]:
svd_model.fit(data)

In [22]:
## normalde verilen değer : rating:4
print(sample_df[sample_df["userId"] == 1])

## kullanıcı bazlı sonuç görüntüleme(est=4.226539781433124 1 nolu kullanıcının 541 nolu filme vereceği puan tahmini)
svd_model.predict(uid=1.0, iid=541, verbose=True)

        movieId                title                  genres  userId  rating  \
664545      541  Blade Runner (1982)  Action|Sci-Fi|Thriller     1.0     4.0   

                  timestamp  
664545  2005-04-02 23:30:03  
user: 1.0        item: 541        r_ui = None   est = 4.23   {'was_impossible': False}


Prediction(uid=1.0, iid=541, r_ui=None, est=4.226539781433124, details={'was_impossible': False})